In [ ]:
#default_exp synchro.extracting

In [ ]:
#export
import numpy as np
import datetime
import os, glob

class QDSpy_log:
    def __init(self, log_dir):
        self.dir = log_dir
        self.log_names = glob.glob(os.path.join(self.dir,'[0-9]*.log'))
        self.stimuli = []
        self.comments = []
        
    def _extract_data(self, data_line):
        data = data_line[data_line.find('{')+1:data_line.find('}')]
        data_splitted = data.split(',')
        data_dict = {}
        for data in data_splitted:
            ind = data.find("'")
            if type(data[data.find(":")+2:]) is str:
                data_dict[data[ind+1:data.find("'",ind+1)]] = data[data.find(":")+2:][1:-1]
            else:
                data_dict[data[ind+1:data.find("'",ind+1)]] = data[data.find(":")+2:]
        return data_dict

    def _extract_time(self,data_line):
        line = '%s' % data_line
        year = int(line[0:4])
        month = int(line[4:6])
        day = int(line[6:8])
        hour =int(line[9:11])
        minute = int(line[11:13])
        second = int(line[13:15])
        result = datetime.datetime(year,month,day,hour,minute,second)
        return result
    
    def _extract_dropped(self,data_line):
        ind = data_line.find('#')
        index_frame = int(data_line[ind+1:data_line.find(' ',ind)])
        ind = data_line.find('was')
        delay = float(data_line[ind:].split(" ")[1])
        return (index_frame, delay)
    
    @property
    def n_stim(self):
        n_stim = 0
        for s in stimuli:
            n_stim+=len(s)
        return n_stim

    @property
    def stim_names(self):
        return [stim.name for stim in stimuli for stimuli in self.stimuli]
    
    def find_stimuli(self):
        """Find the stimuli in the log file and return the list of the stimuli
        found by this object."""
        for k, log_name in enumerate(self.log_names):
            self.stimuli.append([])
            with open(os.path.join(self.dir, log_name), 'r') as log_file:
                for line in log_file:
                    if "DATA" in line:
                        data_juice = self._extract_data(line)
                        if 'stimState' in data_juice.keys():
                            if data_juice['stimState'] == "STARTED" :
                                curr_stim = Stimulus(self._extract_time(line))
                                curr_stim.set_parameters(data_juice)
                                self.stimuli[k].append(curr_stim)
                                stimulus_ON = True
                            elif data_juice['stimState'] == "FINISHED" or data_juice['stimState'] == "ABORTED":
                                curr_stim.is_aborted = data_juice['stimState'] == "ABORTED"
                                curr_stim.stop_time = self._extract_time(line)
                                stimulus_ON = False

                        elif 'userComment' in data_juice.keys():
                            pass
                            #print("userComment, use it to bind logs to records")
                        elif stimulus_ON: #Information on stimulus parameters
                            curr_stim.set_parameters(data_juice)
        #                elif 'probeX' in data_juice.keys():
                #            print("Probe center not implemented yet")
                    if "WARNING" in line and "dt of frame" and stimulus_ON:
                        curr_stim.frame_drop.append(self._extract_dropped(line))
        return self.stimuli

In [ ]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_utils.ipynb.
Converted 02_processing.ipynb.
Converted 03_modelling.ipynb.
Converted 04_plotting.ipynb.
Converted 10_synchro.io.ipynb.


error: bad escape \e at position 150